## Using [Google Earth Engine's](https://earthengine.google.com/) (GEE) [Python API](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api)

Earth Engine is a cloud-based tool developed by Google that allows users to interact with and manipulate [numerous datasets](https://developers.google.com/earth-engine/datasets) on the cloud. Their web-based Code Editor, based in JavaScript can be accessed at [code.earthengine.google.com](http://code.earthengine.google.com). They maintain a Python API for easy development with notebooks and Python packages. Users must have a Google Cloud Platform (GCP) [cloud project to use GEE](https://developers.google.com/earth-engine/guides/transition_to_cloud_projects) though noncommercial use is free.

Learn more: Gorelick, N., Hancher, M., Dixon, M., Ilyushchenko, S., Thau, D., & Moore, R. (2017). Google Earth Engine: Planetary-scale geospatial analysis for everyone. Remote Sensing of Environment, 202, 18-27. https://doi.org/10.1016/j.rse.2017.06.031

## Getting Started

In [1]:
import ee
import geemap

Earth Engine requires authentication to download and manipulate data using their API. The cell below will run their authentication and initialization procedure. You will need to authenticate by pasting a token generated using the authentication link.

In [2]:
ee.Authenticate()
ee.Initialize()

Next, we can select a dataset and create a query for an image collection, or an EE object of mutiple satellite images. In this example, we will query for Antarctic imagery from Landsat-7 around Dome A captured during 2002.

In [11]:
# Create collection of images from GrIMP dataset at Zachariæ Isstrøm (ZI) during 2014
col = (ee.ImageCollection('OSU/GIMP/ICE_VELOCITY_OPT') # 
         .filterDate('2014-01-01', '2014-12-31') # Filter for results from 2014
)

In [15]:
dem = ee.Image('OSU/GIMP/DEM').select('elevation') # Load the Digital Elevation Model (DEM) from the same dataset

In [21]:
mosaic = ee.Image('OSU/GIMP/2000_IMAGERY_MOSAIC')

In [22]:
iceVelocityVis = {
    'min': [-1000],
    'max': [1000],
    'palette': ['red', 'black', 'blue'],
    'bands': ['velocity_x']
}
elevationVis = {
    'min': 0,
    'max': 2000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}
moisaicVis = {
    'min': 0,
    'max': 255,
    'bands': ['B1', 'B2', 'B3']
}

In [23]:
Map = geemap.Map()
# Map.addLayer(dem, elevationVis)
Map.addLayer(mosaic, moisaicVis, 'Mosaic')
Map.setCenter(-41.0, 76.0, 4)
# Map.addLayer(col, iceVelocityVis, 'Landsat 7 images at Dome A')
Map

Map(center=[76.0, -41.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

Here we can export the collection to Google Drive to download.

In [ ]:
# Export the image, specifying the CRS, transform, and region.
roi = ee.Geometry.Rectangle([-92314.86136242, -330000, 1657682.84194774, 719092.6152132], proj='EPSG:3031', evenOdd=False)

export = col.select('R47')
task = ee.batch.Export.image.toDrive(
    image=export,
    description='r47_mean_2002',
    crs='EPSG:3031',
    region=roi, # Be sure to change this as needed
    maxPixels=2170754117812,
    scale=100
)
task.start()